# 0. Getting start with OPEN3D Lib

In [1]:
import open3d as o3d
import numpy as np
import code,os,sys,struct    
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
LandingPath = os.path.join(r'KMOU_new_data\Landing','20220921_161009_856_S1.ply')

def plot_raw_pcl(LandingPath):
    pcd = o3d.io.read_point_cloud(LandingPath)
    # print(np.asarray(pcd.points))
    # o3d.visualization.draw_geometries([pcd])
    o3d.visualization.draw_geometries([pcd],
                                    width=1280,
                                    height=860,
                                    zoom=0.3,
                                    front=[ -0.028201869900951016, 0.22625896174693114, -0.97365884002729142 ],
                                    lookat=[ -2780.6908134776941, -590.79499333057709, 959.06654143345077 ],
                                    up=[ 0.042813262481875822, 0.97342585230251621, 0.22496474084793408 ])

    # vis = o3d.visualization.Visualizer()
    # vis.create_window(width=1360,height=900)
    # vis.add_geometry(pcd)
    # ctr  = vis.get_view_control()
    # ctr.rotate(1190,0)
    # vis.run()
    # vis.destroy_window()

if __name__ == "__main__":
    plot_raw_pcl(LandingPath)                            

# 1. Crop Bounding Box

In [13]:
# def CropPCLBox():
#     pcd = o3d.io.read_point_cloud(LandingPath)
#     corners = np.array([
#     [0, 0, 0],
#     [1, 0, 0],
#     [0, 1, 0],
#     [1, 1, 0],
#     [0, 0, 1],
#     [1, 0, 1],
#     [0, 1, 1],
#     [1, 1, 1]])

#     # Convert the corners array to have type float64
#     bounding_polygon = corners.astype("float64")

#     # Create a SelectionPolygonVolume
#     vol = o3d.visualization.SelectionPolygonVolume()

#     # You need to specify what axis to orient the polygon to.
#     # I choose the "Y" axis. I made the max value the maximum Y of
#     # the polygon vertices and the min value the minimum Y of the
#     # polygon vertices.
#     vol.orthogonal_axis = "Y"
#     vol.axis_max = np.max(bounding_polygon[:, 1])
#     vol.axis_min = np.min(bounding_polygon[:, 1])

#     # Set all the Y values to 0 (they aren't needed since we specified what they
#     # should be using just vol.axis_max and vol.axis_min).
#     bounding_polygon[:, 1] = 0

#     # Convert the np.array to a Vector3dVector
#     vol.bounding_polygon = o3d.utility.Vector3dVector(bounding_polygon)

#     # Crop the point cloud using the Vector3dVector
#     cropped_pcd = vol.crop_point_cloud(pcd)

#     # Get a nice looking bounding box to display around the newly cropped point cloud
#     # (This part is optional and just for display purposes)
#     bounding_box = cropped_pcd.get_axis_aligned_bounding_box()
#     bounding_box.color = (1, 0, 0)

#     # Draw the newly cropped PCD and bounding box
#     o3d.visualization.draw_geometries([cropped_pcd])
# if __name__ == "__main__":
#     CropPCLBox()

# 2. Pass Through Filter

In [3]:
class Filter3D_Passthrough():
    def __init__(self, is_rgb_4byte = False):
        self.is_rgb_4byte = is_rgb_4byte
        

    def rgb2float( self,r, g, b, a = 0 ):
        return struct.unpack('f', struct.pack('i',r << 16 | g << 8 | b))[0]

    def draw_guild_lines(self,dic, density = 0.5):
        new_col = []
        new_pos = []
        x_start,x_end = dic["x"]
        y_start,y_end = dic["y"]
        z_start,z_end = dic["z"]

        x_points,y_points,z_points = np.asarray(np.arange(x_start,x_end,density)),np.asarray(np.arange(y_start,y_end,density)),np.asarray(np.arange(z_start,z_end,density))
        
        y_starts,y_ends = np.asarray(np.full((len(x_points)),y_start)),np.asarray(np.full((len(x_points)),y_end))
        z_starts,z_ends = np.asarray(np.full((len(x_points)),z_start)),np.asarray(np.full((len(x_points)),z_end))
        lines_x = np.concatenate((np.vstack((x_points,y_starts,z_starts)).T,np.vstack((x_points,y_ends,z_starts)).T,np.vstack((x_points,y_starts,z_ends)).T,np.vstack((x_points,y_ends,z_ends)).T))


        x_starts,x_ends = np.asarray(np.full((len(y_points)),x_start)),np.asarray(np.full((len(y_points)),x_end))
        z_starts,z_ends = np.asarray(np.full((len(y_points)),z_start)),np.asarray(np.full((len(y_points)),z_end))
        lines_y = np.concatenate((np.vstack((x_starts,y_points,z_starts)).T,np.vstack((x_ends,y_points,z_starts)).T,np.vstack((x_starts,y_points,z_ends)).T,np.vstack((x_ends,y_points,z_ends)).T))


        x_starts,x_ends = np.asarray(np.full((len(z_points)),x_start)),np.asarray(np.full((len(z_points)),x_end))
        y_starts,y_ends = np.asarray(np.full((len(z_points)),y_start)),np.asarray(np.full((len(z_points)),y_end))
        lines_z = np.concatenate((np.vstack((x_starts,y_starts,z_points)).T,np.vstack((x_ends,y_starts,z_points)).T,np.vstack((x_starts,y_ends,z_points)).T,np.vstack((x_ends,y_ends,z_points)).T))

        if (self.is_rgb_4byte):
            lines_x_color =  np.full((len(lines_x)),self.rgb2float(255,0,0))#blue for x
            lines_y_color =  np.full((len(lines_y)),self.rgb2float(0,255,0))#green for y
            lines_z_color =  np.full((len(lines_z)),self.rgb2float(0,0,255))#red for z
            return np.concatenate((lines_x,lines_y,lines_z)),np.asmatrix(np.concatenate((lines_x_color,lines_y_color,lines_z_color))).T
        else:
            lines_x_color = np.zeros((len(lines_x),3))
            lines_y_color = np.zeros((len(lines_y),3))
            lines_z_color = np.zeros((len(lines_z),3))

            lines_x_color[:,0] = 1.0 #red for x
            lines_y_color[:,1] = 1.0 #green for y
            lines_z_color[:,2] = 1.0 #blue for z
            return np.concatenate((lines_x,lines_y,lines_z)),np.asmatrix(np.concatenate((lines_x_color,lines_y_color,lines_z_color)))
    
    def pass_through_filter(self, dic, pcd):

        points = np.asarray(pcd.points)
        colors = np.asarray(pcd.colors)
        points[:,0]
        x_range = np.logical_and(points[:,0] >= dic["x"][0] ,points[:,0] <= dic["x"][1])
        y_range = np.logical_and(points[:,1] >= dic["y"][0] ,points[:,1] <= dic["y"][1])
        z_range = np.logical_and(points[:,2] >= dic["z"][0] ,points[:,2] <= dic["z"][1])

        pass_through_filter = np.logical_and(x_range,np.logical_and(y_range,z_range))

        pcd.points = o3d.utility.Vector3dVector(points[pass_through_filter])
        pcd.paint_uniform_color([0.25, 0.62, 1])
        # pcd.colors = o3d.utility.Vector3dVector(colors[pass_through_filter])

        return pcd


if __name__ == "__main__":
    pcd = o3d.io.read_point_cloud(LandingPath)
    pcd.paint_uniform_color([0.25, 0.62, 1])

    dic = {"x":[-1000,-200],
            "y":[-1000,1000],
            "z":[1000,2500]}
    
    filPT = Filter3D_Passthrough()

    #Drawing filter guidelines 
    new_pos, new_col = filPT.draw_guild_lines(dic)
    new_data = np.concatenate((new_pos, new_col),axis = 1)
    guild_points = o3d.geometry.PointCloud()
    guild_points.points = o3d.utility.Vector3dVector(new_pos)
    guild_points.colors = o3d.utility.Vector3dVector(new_col)

    #Pass through filtering
    pcd_filtered = filPT.pass_through_filter(dic,pcd)

    # o3d.visualization.draw_geometries([guild_points,pcd])
    o3d.visualization.draw_geometries([guild_points,pcd],
                                    width=1280,
                                    height=860,
                                    zoom=0.74,
                                    front=[ -0.35237385107751878, 0.16213571919318584, -0.92170747943070697 ],
                                    lookat=[ -622.15540635158163, -106.14283325914268, 1558.9631819212902 ],
                                    up=[ 0.038047202656848432, 0.98654588887774564, 0.15899565877220476 ])

    

# 3. Statistical and Radius outliner removals (or filter)

In [ ]:
# from ast import increment_lineno


def prepare_raw_pcl(LandingPath):
    # pcd = o3d.io.read_point_cloud(LandingPath)

    voxel_down_pcd = pcd.voxel_down_sample(voxel_size = 0.01)
    # voxel_down_pcd.paint_uniform_color([0.25, 0.62, 1])
    ### Visualization the raw point cloud
    o3d.visualization.draw_geometries([voxel_down_pcd],
                                    width=1280,
                                    height=860,
                                    zoom=0.74,
                                    front=[ -0.35237385107751878, 0.16213571919318584, -0.92170747943070697 ],
                                    lookat=[ -622.15540635158163, -106.14283325914268, 1558.9631819212902 ],
                                    up=[ 0.038047202656848432, 0.98654588887774564, 0.15899565877220476 ])
    return voxel_down_pcd

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    ### Visualization the filterd point cloud
    o3d.visualization.draw_geometries([inlier_cloud],
                                    width=1280,
                                    height=860,
                                    zoom=0.74,
                                    front=[ -0.35237385107751878, 0.16213571919318584, -0.92170747943070697 ],
                                    lookat=[ -622.15540635158163, -106.14283325914268, 1558.9631819212902 ],
                                    up=[ 0.038047202656848432, 0.98654588887774564, 0.15899565877220476 ])
    FilteredPath = os.path.join(r'3D Camera\Dataset\Filtered Data\PaT_StR_RaR','20221026_Landing_1.ply')
    o3d.io.write_point_cloud(FilteredPath,inlier_cloud)
    
    ### Visulization the eleminated point cloud
    outlier_cloud.paint_uniform_color([1, 0, 0])
    o3d.visualization.draw_geometries([outlier_cloud],
                                    width=1280,
                                    height=860,
                                    zoom=0.74,
                                    front=[ -0.35237385107751878, 0.16213571919318584, -0.92170747943070697 ],
                                    lookat=[ -622.15540635158163, -106.14283325914268, 1558.9631819212902 ],
                                    up=[ 0.038047202656848432, 0.98654588887774564, 0.15899565877220476 ])

    ### Visulization the both filterd and eleminated point cloud
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                    width=1280,
                                    height=860,
                                    zoom=0.74,
                                    front=[ -0.35237385107751878, 0.16213571919318584, -0.92170747943070697 ],
                                    lookat=[ -622.15540635158163, -106.14283325914268, 1558.9631819212902 ],
                                    up=[ 0.038047202656848432, 0.98654588887774564, 0.15899565877220476 ])

def statistical_outlier_removal(voxel_down_pcd):
    print("Statistical outlier removal")
    cl, ind = voxel_down_pcd.remove_statistical_outlier(nb_neighbors=20,
                                                        std_ratio=0.2)
    # display_inlier_outlier(voxel_down_pcd, ind)
    return voxel_down_pcd.select_by_index(ind)

def radius_outlier_removal(voxel_down_pcd):
    print("Radius outlier removal")
    cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=8, radius=10)
    display_inlier_outlier(voxel_down_pcd, ind)
    return voxel_down_pcd.select_by_index(ind)

if __name__ == "__main__":
    LandingPath = os.path.join(r'3D Camera\Dataset\KMOU_new_data\Landing','20220921_161009_856_S1.ply')
    preprocess_pcl = prepare_raw_pcl(LandingPath)
    filtered_sta_out_remove = statistical_outlier_removal(preprocess_pcl)
    Filtered_PCL = radius_outlier_removal(filtered_sta_out_remove)          

# 4. Clustering / Segmentation / Recognition / Classification

In [5]:
##### DBSCAN CLUSTERING 
def dbscan_cluster():
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            Filtered_PCL.cluster_dbscan(eps=15, min_points=40, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    Filtered_PCL.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([Filtered_PCL],
                                        width=1280,
                                        height=860,
                                        zoom=0.52,
                                        front=[ 0.35841336406485025, 0.1045412555722367, -0.92769121281981404 ],
                                        lookat=[ -572.38828795693564, -374.20006974981737, 1582.1566409846967 ],
                                        up=[ -0.076723071324817052, 0.99364733736702315, 0.082331884892261439 ])
    return labels

if __name__ == "__main__":
    labels_cluster = dbscan_cluster()

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 22
point cloud has 22 clusters


In [6]:
##### Select clusters from DBScan clustering
def draw_boundingbox_casting(cloud_pcd):
    dic = {"x":[0,0],
            "y":[0,0],
            "z":[0,0]}
    max_bound = o3d.geometry.OrientedBoundingBox.get_max_bound(cloud_pcd)
    min_bound = o3d.geometry.OrientedBoundingBox.get_min_bound(cloud_pcd)
    i = 0
    for eles in dic:
        if eles == 'x':
            dic[eles][0] = min_bound[i]+350
            dic[eles][1] = max_bound[i]
        elif eles == 'y':
            dic[eles][0] = min_bound[i]
            dic[eles][1] = max_bound[i]-500
        elif eles == 'z':
            dic[eles][0] = min_bound[i]
            dic[eles][1] = max_bound[i]
        i+=1

    new_pos, new_col = filPT.draw_guild_lines(dic)
    new_data = np.concatenate((new_pos, new_col),axis = 1)
    guild_points = o3d.geometry.PointCloud()
    guild_points.points = o3d.utility.Vector3dVector(new_pos)
    guild_points.colors = o3d.utility.Vector3dVector(new_col)
    return guild_points

def draw_boundingbox_twl(cloud_pcd):
    dic = {"x":[0,0],
            "y":[0,0],
            "z":[0,0]}
    max_bound = o3d.geometry.OrientedBoundingBox.get_max_bound(cloud_pcd)
    min_bound = o3d.geometry.OrientedBoundingBox.get_min_bound(cloud_pcd)
    i = 0
    for eles in dic:
        dic[eles][0] = min_bound[i]
        dic[eles][1] = max_bound[i]
        i+=1

    new_pos, new_col = filPT.draw_guild_lines(dic)
    new_data = np.concatenate((new_pos, new_col),axis = 1)
    guild_points = o3d.geometry.PointCloud()
    guild_points.points = o3d.utility.Vector3dVector(new_pos)
    guild_points.colors = o3d.utility.Vector3dVector(new_col)
    return guild_points

def select_cluster(labels_cluster):
    unique, counts = np.unique(labels_cluster, return_counts=True)
    labels_dict = dict(zip(unique, counts))
    
    ### Choose point cloud of container corner casting
    ind_cornercasting = np.where(labels_cluster == max(labels_dict, key=labels_dict.get))[0]
    cornercasting_pcd = Filtered_PCL.select_by_index(ind_cornercasting)
    casting_maxbound = o3d.geometry.OrientedBoundingBox.get_max_bound(cornercasting_pcd)
    casting_minbound = o3d.geometry.OrientedBoundingBox.get_min_bound(cornercasting_pcd)
    
    ### Choose point cloud of twist lock
    sorted_volume = np.argsort(counts)          ###Return the indices of the sorted value 
    sorted_volume = sorted_volume[::-1]         ###Reverse the order
    twistlock_pcd = Filtered_PCL.select_by_index(ind_cornercasting, invert=True)
    for i in range(3):
        i+=1
        ind_twl = np.where(labels_cluster==sorted_volume[i]-1)[0]
        if sorted_volume[i]-1 != -1:            ###Eleminate the noise of the filtered point clouds
            temp_twl = Filtered_PCL.select_by_index(ind_twl)
            temp_maxbound = o3d.geometry.OrientedBoundingBox.get_max_bound(temp_twl)
            temp_minbound = o3d.geometry.OrientedBoundingBox.get_min_bound(temp_twl)
            if temp_maxbound[0] <= casting_maxbound[0]+100 and temp_maxbound[2] <= casting_maxbound[2]+100:
                twistlock_pcd = temp_twl

    ### Draw bounding of the objects (Container corner casting and Twist lock)
    bbox_casting = draw_boundingbox_casting(cornercasting_pcd)
    bbox_twl = draw_boundingbox_twl(twistlock_pcd)

    ### Visualization the result
    o3d.visualization.draw_geometries([cornercasting_pcd, bbox_casting,twistlock_pcd, bbox_twl],
                                        width=1280,
                                        height=860,
                                        zoom=0.88,
                                        front=[ 0.31767370639682024, 0.15340804496800778, -0.93570796085274421 ],
                                        lookat=[ -612.88317056989149, -296.48142700345363, 1572.524502752294 ],
                                        up=[ -0.09149002627593722, 0.98718013848535224, 0.13078589095185073 ])
    RstPath = os.path.join(r'3D Camera\Dataset\Result Data\dbscan','20221027_Landing_1_rs.ply')
    rst_detection = o3d.geometry.PointCloud()
    # rst_detection.points = o3d.utility.Vector3dVector
    # o3d.io.write_point_cloud(RstPath,rst_detection)
    return rst_detection
if __name__ == "__main__":
    rst_detection = select_cluster(labels_cluster)